#### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log1p(y_pred) + (1.0 - y) * np.log1p(1.0 - y_pred))
    return err

In [3]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

-0.6148202755372569

In [4]:
# Граничный пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.2])
calc_logloss(y1, y_pred1)

-0.6404669227310322

#### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [5]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [6]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [7]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [8]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [9]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [10]:
W = eval_model(X_st, y, iterations=500, eta=1e-6)

0 [ 0.49671377 -0.13826575  0.64768851  1.52302929] -0.40567765245791315
50 [ 0.4966948  -0.13833829  0.64768714  1.52300098] -0.4056858926470109
100 [ 0.49667584 -0.13841083  0.64768577  1.52297268] -0.405694133267087
150 [ 0.49665687 -0.13848337  0.6476844   1.52294437] -0.4057023743177658
200 [ 0.4966379  -0.1385559   0.64768303  1.52291607] -0.4057106157986722
250 [ 0.49661894 -0.13862843  0.64768166  1.52288777] -0.4057188577094307
300 [ 0.49659997 -0.13870096  0.64768029  1.52285947] -0.4057271000496664
350 [ 0.49658101 -0.13877348  0.64767891  1.52283117] -0.405735342819004
400 [ 0.49656205 -0.138846    0.64767754  1.52280287] -0.40574358601706856
450 [ 0.49654309 -0.13891852  0.64767617  1.52277457] -0.40575182964348533


#### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [11]:
def calc_pred_proba(W, X):
    z = sigmoid(np.dot(X,W))
    return z

In [12]:
W = eval_model(X_st, y, iterations=100000, eta=0.5)

0 [ 0.30702071 -0.8637023   0.63399833  1.23995672] -0.40567765245791315
10000 [-24.52908728  -2.59446649  -6.46646026  20.31481166] -0.6290668687355121
20000 [-33.59529145  -3.41367073  -9.03407815  27.6566593 ] -0.6418466106595424
30000 [-39.81744841  -3.96631789 -10.78976588  32.6585122 ] -0.647736463148566
40000 [-44.75906464  -4.39229709 -12.19195091  36.59812039] -0.6513774997827542
50000 [-48.98128881  -4.74319844 -13.40056249  39.93475083] -0.6539924907436797
60000 [-52.74597263  -5.04366664 -14.48880326  42.88347092] -0.656045320580432
70000 [-56.1947235   -5.30759987 -15.49541444  45.56162416] -0.6577521213817646
80000 [-59.41142516  -5.54385926 -16.44267207  48.03989384] -0.6592276106220333
90000 [-62.44881255  -5.75862516 -17.34403189  50.3638311 ] -0.6605382843150149


In [13]:
calc_pred_proba(W, X_st)

array([2.89578406e-01, 2.23157729e-04, 1.00000000e+00, 4.28637591e-13,
       9.83394794e-01, 2.01330849e-03, 1.00000000e+00, 3.32748084e-03,
       7.27120422e-01, 1.00000000e+00])

#### 4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [14]:
def calc_pred(W, X, treshold = 0.5):
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > treshold, 1, 0 )
    return y_pred

In [15]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

#### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [24]:
def get_results(y, y_pred):
    TP = ((y == 1) & (y_pred == 1)).sum() 
    TN = ((y == 0) & (y_pred == 0)).sum()
    FN = ((y == 1) & (y_pred == 0)).sum() 
    FP = ((y == 0) & (y_pred == 1)).sum() 
    
    accuracy = (TP + TN) / (TP + TN + FN + FP)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * precision * recall / precision * recall
    
    return ((TP, TN, FN, FP), accuracy, precision, recall, f1_score)

In [25]:
(y, y_pred)


(array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]),
 array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]))

#### 6. Могла ли модель переобучиться? Почему?

In [ ]:
В данном случае у нас очень мало наблюдей.